In [4]:
import os
import ipywidgets as widgets
from IPython.display import display

# Define an ignore list
ignore_list = [
    '.git',          # Ignore the .git directory
    'ignore_this',   # Add any additional files or directories to ignore
    'another_ignore',
    '.vscode',       # Ignore the .vscode directory
    'Archive',
    'Examples',
    'utilities',
]

def should_ignore(path):
    """Check if the path should be ignored based on the ignore list."""
    # Check against the ignore list
    for pattern in ignore_list:
        if pattern in path:
            return True
    return False

def list_directories(start_path):
    """Recursively list directories starting from the given path."""
    directory_list = []
    
    def add_directories(path, level=0):
        # Check if the current directory should be ignored
        if os.path.basename(path) in ignore_list:
            return
        
        # Add the current path to the list with indentation based on level
        directory_list.append(' ' * level * 4 + os.path.basename(path))
        
        for entry in os.listdir(path):
            full_path = os.path.join(path, entry)
            if os.path.isdir(full_path):
                add_directories(full_path, level + 1)

    add_directories(start_path)
    return directory_list

# Specify the starting directory (current working directory)
start_directory = os.getcwd()
directories = list_directories(start_directory)

# Create a dropdown widget with a wider label
directory_dropdown = widgets.Dropdown(
    options=directories,
    description='Select Directory:',
    disabled=False,
    style={'description_width': 'initial'}  # Adjust label width
)

# Display the dropdown
display(directory_dropdown)


Dropdown(description='Select Directory:', options=('emotional_analysis', '    all_plots', '        240930_1613…

In [12]:
import mne

def split_raw_into_intervals(raw, original_filename, base_output_folder, interval=2):
    # Get the number of samples per interval
    sfreq = raw.info['sfreq']  # Sampling frequency
    n_samples = int(sfreq * interval)

    # Extract base name without extension
    base_name = os.path.splitext(original_filename)[0]  

    # Create a subfolder named after the first 6 characters of the original filename
    subfolder_name = base_name[:6]  # Get the first 6 characters
    output_folder = os.path.join(base_output_folder, 'splif_fifs', subfolder_name)
    os.makedirs(output_folder, exist_ok=True)

    for start in range(0, len(raw.times), n_samples):
        # Get the end of the interval
        end = start + n_samples
        
        # Ensure the end does not exceed the maximum time
        if end > len(raw.times):  # Handle the last interval
            end = len(raw.times)

        # Calculate tmin and tmax in seconds
        tmin = start / sfreq
        tmax = (end / sfreq) - 0.1

        # Extract the data for the current interval
        try:
            interval_data = raw.copy().crop(tmin=tmin, tmax=tmax)
        except Exception as e:
            interval_data = raw.copy().crop(tmin=tmin, tmax=raw.times[-1])

        try:
            # Create a new filename that ends with raw.fif
            interval_filename = f"{base_name}_interval_{start // n_samples}.raw.fif"
            
            # Save the interval data to the specified output folder
            interval_data.save(os.path.join(output_folder, interval_filename), overwrite=True)
        except Exception as e:
            print(f'Failed to save: {e}')
        
        print(f"Saved: {os.path.join(output_folder, interval_filename)}")

def process_fif_files_in_directory(directory):
    # Create the main output folder
    base_output_folder = directory
    os.makedirs(os.path.join(base_output_folder, 'splif_fifs'), exist_ok=True)

    # Walk through the directory recursively
    for root, dirs, files in os.walk(directory):
        # Ignore specified directories
        dirs[:] = [d for d in dirs if not should_ignore(d)]  # Modify in-place

        for file in files:
            # Ignore specified files based on the ignore list
            if should_ignore(file):
                print(f"Ignoring file: {file}")
                continue
            
            if file.endswith('.fif'):
                file_path = os.path.join(root, file)
                print(f"Processing: {file_path}")
                raw = mne.io.read_raw_fif(file_path, preload=True)  # Load the raw data
                split_raw_into_intervals(raw, file, base_output_folder)  # Pass the base output folder

# Run this after selecting the directory
selected_directory = directory_dropdown.value

if selected_directory:
    # Remove leading spaces to get the correct path
    selected_directory_path = os.path.join(os.getcwd(), selected_directory.strip())
    process_fif_files_in_directory(selected_directory_path)
else:
    print("Please select a directory first.")


Processing: c:\Users\mw24396\OneDrive - Georgia Southern University\1_EEG\1_ Senior_Capstone\Emotional_Analysis\emotional_analysis\all_plots\240930_161302mt_0.6eogt_4db_True_nrmlizd_True_cmp_5\103918\103918raw.fif
Opening raw data file c:\Users\mw24396\OneDrive - Georgia Southern University\1_EEG\1_ Senior_Capstone\Emotional_Analysis\emotional_analysis\all_plots\240930_161302mt_0.6eogt_4db_True_nrmlizd_True_cmp_5\103918\103918raw.fif...
    Range : 0 ... 143999 =      0.000 ...  1124.992 secs
Ready.
Reading 0 ... 143999  =      0.000 ...  1124.992 secs...
Writing c:\Users\mw24396\OneDrive - Georgia Southern University\1_EEG\1_ Senior_Capstone\Emotional_Analysis\emotional_analysis\all_plots\splif_fifs\103918\103918raw_interval_0.raw.fif
Closing c:\Users\mw24396\OneDrive - Georgia Southern University\1_EEG\1_ Senior_Capstone\Emotional_Analysis\emotional_analysis\all_plots\splif_fifs\103918\103918raw_interval_0.raw.fif
[done]
Saved: c:\Users\mw24396\OneDrive - Georgia Southern University\

KeyboardInterrupt: 

In [ ]:
import os
import mne

def split_raw_into_intervals(raw, original_filename, base_output_folder, interval=2):
    # Get the number of samples per interval
    sfreq = raw.info['sfreq']  # Sampling frequency
    n_samples = int(sfreq * interval)

    # Extract base name without extension
    base_name = os.path.splitext(original_filename)[0]  

    # Create a subfolder named after the first 6 characters of the original filename
    subfolder_name = base_name[:6]  # Get the first 6 characters
    output_folder = os.path.join(base_output_folder, 'splif_fifs', subfolder_name)
    os.makedirs(output_folder, exist_ok=True)

    for start in range(0, len(raw.times), n_samples):
        # Get the end of the interval
        end = start + n_samples
        
        # Ensure the end does not exceed the maximum time
        if end > len(raw.times):  # Handle the last interval
            end = len(raw.times)

        # Calculate tmin and tmax in seconds
        tmin = start / sfreq
        tmax = (end / sfreq) - 0.1

        # Extract the data for the current interval
        try:
            interval_data = raw.copy().crop(tmin=tmin, tmax=tmax)
        except Exception as e:
            interval_data = raw.copy().crop(tmin=tmin, tmax=raw.times[-1])

        try:
            # Create a new filename that ends with raw.fif
            interval_filename = f"{base_name}_interval_{start // n_samples}.raw.fif"
            
            # Save the interval data to the specified output folder
            interval_data.save(os.path.join(output_folder, interval_filename), overwrite=True)
        except Exception as e:
            print(f'Failed to save: {e}')
        
        print(f"Saved: {os.path.join(output_folder, interval_filename)}")

def process_fif_files_in_directory(directory):
    # Create the main output folder
    base_output_folder = directory
    os.makedirs(os.path.join(base_output_folder, 'splif_fifs'), exist_ok=True)

    # Walk through the directory recursively
    for root, dirs, files in os.walk(directory):
        for file in files:
            # Check if the file should be ignored
            if file.endswith('.fif') and len(file) <= 13 and file[:6].isalnum():
                print(f"Ignoring file: {file}")
                continue
            
            if file.endswith('.fif'):
                file_path = os.path.join(root, file)
                print(f"Processing: {file_path}")
                raw = mne.io.read_raw_fif(file_path, preload=True)  # Load the raw data
                split_raw_into_intervals(raw, file, base_output_folder)  # Pass the base output folder

# Specify the directory containing .fif files
directory_path = r'C:\\Users\\mw24396\\OneDrive - Georgia Southern University\\1_EEG\\1_ Senior_Capstone\\Emotional_Analysis\\emotional_analysis\\all_plots\\240930_161302mt_0.6eogt_4db_True_nrmlizd_True_cmp_5'
process_fif_files_in_directory(directory_path)
